# PYSPARK [Aggregate fuctions, Joins, Spark Sql] NOV 21 2024

# Manipulating, Droping, Sorting, Aggregations, Joining, GroupeBy DataFrames

## working with sample data

### create data

In [0]:
from pyspark.sql import SparkSession


# Initialize SparkSession
spark = SparkSession.builder \
.appName("example") \
.getOrCreate()
# Data
simpleData = [("James","Sales","NY",90000,34,10000),
("Michael","Sales","NY",86000,56,20000),
("Robert","Sales","CA",81000,30,23000),
("Maria","Finance","CA",90000,24,23000),
("Raman","Finance","CA",99000,40,24000),
("Scott","Finance","NY",83000,36,19000),
("Jen","Finance","NY",79000,53,15000),
("Jeff","Marketing","CA",80000,25,18000),
("Kumar","Marketing","NY",91000,50,21000)
]
# Create DataFrame
schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



### groupby()

In [0]:
# groupby with sum of salaries
sumdata = df.groupBy("department").sum("salary")
sumdata.show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|     257000|
|   Finance|     351000|
| Marketing|     171000|
+----------+-----------+



#### groupby with single column

In [0]:
df.groupBy("department").min("salary").show()
df.groupBy("department").max("salary").show()
df.groupBy("department").avg("salary").show()
df.groupBy("department").mean("salary").show()
df.groupBy("department").count().show()

+----------+-----------+
|department|min(salary)|
+----------+-----------+
|     Sales|      81000|
|   Finance|      79000|
| Marketing|      80000|
+----------+-----------+

+----------+-----------+
|department|max(salary)|
+----------+-----------+
|     Sales|      90000|
|   Finance|      99000|
| Marketing|      91000|
+----------+-----------+

+----------+-----------------+
|department|      avg(salary)|
+----------+-----------------+
|     Sales|85666.66666666667|
|   Finance|          87750.0|
| Marketing|          85500.0|
+----------+-----------------+

+----------+-----------------+
|department|      avg(salary)|
+----------+-----------------+
|     Sales|85666.66666666667|
|   Finance|          87750.0|
| Marketing|          85500.0|
+----------+-----------------+

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+



#### grouping with multiple columns

In [0]:
#grouping by multiple columns
df.groupBy("employee_name","department").min("salary").show()
df.groupBy("employee_name","department").max("salary").show()
df.groupBy("employee_name","department").avg("salary").show()
df.groupBy("employee_name","department").mean("salary").show()
df.groupBy("employee_name","department").count().show()

+-------------+----------+-----------+
|employee_name|department|min(salary)|
+-------------+----------+-----------+
|        James|     Sales|      90000|
|      Michael|     Sales|      86000|
|       Robert|     Sales|      81000|
|        Maria|   Finance|      90000|
|        Raman|   Finance|      99000|
|        Scott|   Finance|      83000|
|          Jen|   Finance|      79000|
|         Jeff| Marketing|      80000|
|        Kumar| Marketing|      91000|
+-------------+----------+-----------+

+-------------+----------+-----------+
|employee_name|department|max(salary)|
+-------------+----------+-----------+
|        James|     Sales|      90000|
|      Michael|     Sales|      86000|
|       Robert|     Sales|      81000|
|        Maria|   Finance|      90000|
|        Raman|   Finance|      99000|
|        Scott|   Finance|      83000|
|          Jen|   Finance|      79000|
|         Jeff| Marketing|      80000|
|        Kumar| Marketing|      91000|
+-------------+---------

### pivot()

In [0]:
#using pivot function
df.groupBy("department").sum("salary").show()
df.groupBy("department").pivot("employee_name").sum("salary").show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|     257000|
|   Finance|     351000|
| Marketing|     171000|
+----------+-----------+

+----------+-----+-----+-----+-----+-----+-------+-----+------+-----+
|department|James| Jeff|  Jen|Kumar|Maria|Michael|Raman|Robert|Scott|
+----------+-----+-----+-----+-----+-----+-------+-----+------+-----+
|     Sales|90000| null| null| null| null|  86000| null| 81000| null|
|   Finance| null| null|79000| null|90000|   null|99000|  null|83000|
| Marketing| null|80000| null|91000| null|   null| null|  null| null|
+----------+-----+-----+-----+-----+-----+-------+-----+------+-----+



### dropping null values

In [0]:
from pyspark.sql import SparkSession
# Initialize SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
# Data
simpleData = [("James","Sales","NY",90000,34,10000),
("Michael","Sales","NY",86000,56,20000),("Robert","Sales","CA",81000,None,23000),
("Maria","Finance","CA",90000,24,23000),("Raman","Finance","CA",99000,40,None),
("Scott","Finance","NY",None,36,44000),("Jen","Finance","NY",55000,53,15000),
("Jeff",None,"CA",80000,25,18000),(None,"Marketing","NY",91000,50,21000)]
# Create DataFrame
schema = ["employee_name","department","state","salary","age","bonus"]
dfa = spark.createDataFrame(data=simpleData, schema = schema)
dfa.show()

+-------------+----------+-----+------+----+-----+
|employee_name|department|state|salary| age|bonus|
+-------------+----------+-----+------+----+-----+
|        James|     Sales|   NY| 90000|  34|10000|
|      Michael|     Sales|   NY| 86000|  56|20000|
|       Robert|     Sales|   CA| 81000|null|23000|
|        Maria|   Finance|   CA| 90000|  24|23000|
|        Raman|   Finance|   CA| 99000|  40| null|
|        Scott|   Finance|   NY|  null|  36|44000|
|          Jen|   Finance|   NY| 55000|  53|15000|
|         Jeff|      null|   CA| 80000|  25|18000|
|         null| Marketing|   NY| 91000|  50|21000|
+-------------+----------+-----+------+----+-----+



In [0]:
# Initialize SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
# Data
simpleData = [("James","Sales","NY",90000,34,10000),
("Michael","Sales","NY",86000,56,20000),("Robert","Sales","CA",81000,None,23000),
("Maria","Finance","CA",90000,24,23000),("Raman","Finance","CA",99000,40,None),
("Scott","Finance","NY",None,36,44000),("Jen","Finance","NY",55000,53,15000),
("Jeff",None,"CA",80000,25,18000),(None,"Marketing","NY",91000,50,21000)]
# Create DataFrame
schema = ["employee_name","department","state","salary","age","bonus"]
dfa = spark.createDataFrame(data=simpleData, schema = schema)
dfa.na.drop().show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|          Jen|   Finance|   NY| 55000| 53|15000|
+-------------+----------+-----+------+---+-----+



### salary aggregate sum before and after grouping

In [0]:
df.groupBy("department").agg(({"salary":"sum"})).show()
df.agg(({"salary":"sum"})).show()  # Without group using agg on salary colums

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|     257000|
|   Finance|     351000|
| Marketing|     171000|
+----------+-----------+

+-----------+
|sum(salary)|
+-----------+
|     779000|
+-----------+



## Working with Employee.csv file

### create data

In [0]:
data_emp = spark.read.csv("/FileStore/tables/employee-4.csv",inferSchema=True,header=True)

data_emp.limit(10).display()

EmployeeID,Name,Age,Department,Salary
1,Aarav,59,IT,90000
2,Vivaan,49,HR,60000
3,Aditya,35,IT,100000
4,Anaya,28,null,85000
5,Diya,41,null,110000
6,Krishna,59,Finance,95000
7,Arjun,39,null,85000
8,Ishaan,43,Finance,65000
9,Riya,31,null,95000
10,Saanvi,31,HR,85000


### groupby()

#### single column

In [0]:
data_emp.groupBy("Department").min("salary").show()
data_emp.groupBy("Age").max("salary").show()
data_emp.groupBy("Department").avg("salary").show()
data_emp.groupBy("Age").mean("salary").show()
data_emp.groupBy("Department").count().show()

+----------+-----------+
|Department|min(salary)|
+----------+-----------+
|        HR|      60000|
|      null|      45000|
|   Finance|      35000|
|        IT|      30000|
+----------+-----------+

+---+-----------+
|Age|max(salary)|
+---+-----------+
| 31|      95000|
| 53|      35000|
| 34|      35000|
| 28|      85000|
| 27|      30000|
| 44|      75000|
| 22|      60000|
| 47|      55000|
| 57|      80000|
| 48|      65000|
| 41|     110000|
| 43|      65000|
| 35|     100000|
| 59|      95000|
| 39|      85000|
| 23|      60000|
| 49|      60000|
| 50|     115000|
| 45|      70000|
| 38|      35000|
+---+-----------+
only showing top 20 rows

+----------+-----------------+
|Department|      avg(salary)|
+----------+-----------------+
|        HR|          72500.0|
|      null|82727.27272727272|
|   Finance|64285.71428571428|
|        IT|60384.61538461538|
+----------+-----------------+

+---+-----------------+
|Age|      avg(salary)|
+---+-----------------+
| 31|          90000

#### multiple column

In [0]:
#grouping by multiple columns
data_emp.groupBy("Age","department").min("salary").show()
data_emp.groupBy("Age","department").max("salary").show()
data_emp.groupBy("department","Age").avg("salary").show()
data_emp.groupBy("Name","department").mean("salary").show()
data_emp.groupBy("Age","department").count().show()

+---+----------+-----------+
|Age|department|min(salary)|
+---+----------+-----------+
| 31|      null|      95000|
| 35|        IT|      65000|
| 28|      null|      85000|
| 24|   Finance|      80000|
| 44|      null|       null|
| 29|        IT|     100000|
| 31|        HR|      85000|
| 44|   Finance|      65000|
| 22|      null|      60000|
| 49|        HR|      60000|
| 38|        IT|      35000|
| 27|        IT|      30000|
| 42|        IT|      45000|
| 34|   Finance|      35000|
| 57|      null|      80000|
| 56|   Finance|      55000|
| 59|   Finance|      95000|
| 41|      null|     110000|
| 59|        IT|      90000|
| 44|        IT|      75000|
+---+----------+-----------+
only showing top 20 rows

+---+----------+-----------+
|Age|department|max(salary)|
+---+----------+-----------+
| 31|      null|      95000|
| 35|        IT|     100000|
| 28|      null|      85000|
| 24|   Finance|      80000|
| 44|      null|       null|
| 29|        IT|     100000|
| 31|        HR| 

### dropping null value

In [0]:
data_copy = data_emp
df_clean = data_copy.na.drop()
df_clean.limit(10).display()

EmployeeID,Name,Age,Department,Salary
1,Aarav,59,IT,90000
2,Vivaan,49,HR,60000
3,Aditya,35,IT,100000
6,Krishna,59,Finance,95000
8,Ishaan,43,Finance,65000
10,Saanvi,31,HR,85000
12,Aditi,56,Finance,55000
13,Tanish,44,Finance,65000
15,Priya,42,HR,80000
17,Rohan,44,IT,75000


### groupby after dropping null

In [0]:
#grouping by multiple columns
df_clean.groupBy("Age").max("salary").orderBy("Age").show()
df_clean.groupBy("department","Age").avg("salary").show()
df_clean.groupBy("Age","department").count().show()

+---+-----------+
|Age|max(salary)|
+---+-----------+
| 22|      60000|
| 23|      60000|
| 24|      80000|
| 27|      30000|
| 29|     100000|
| 31|      85000|
| 34|      35000|
| 35|     100000|
| 36|      55000|
| 38|      35000|
| 41|      35000|
| 42|      80000|
| 43|      65000|
| 44|      75000|
| 47|      55000|
| 48|      65000|
| 49|      60000|
| 53|      35000|
| 56|      55000|
| 59|      95000|
+---+-----------+

+----------+---+-----------+
|department|Age|avg(salary)|
+----------+---+-----------+
|        IT| 42|    45000.0|
|        IT| 59|    90000.0|
|        IT| 22|    60000.0|
|        HR| 42|    80000.0|
|   Finance| 59|    95000.0|
|        IT| 23|    60000.0|
|   Finance| 56|    55000.0|
|        HR| 48|    65000.0|
|        IT| 41|    35000.0|
|        IT| 29|   100000.0|
|        IT| 38|    35000.0|
|   Finance| 43|    65000.0|
|        HR| 31|    85000.0|
|        IT| 53|    35000.0|
|   Finance| 24|    80000.0|
|        IT| 47|    55000.0|
|   Finance| 44|

## Working with Loan Data

### Create data

In [0]:
data_loan = spark.read.csv("/FileStore/tables/LoanData-2.csv",inferSchema=True,header=True)

### data exploration

In [0]:
data_loan.limit(10).display()

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001002,Male,No,0,Graduate,No,5849,0.0,null,360,1,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95,360,1,Urban,Y
LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158,360,0,Semiurban,N
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168,360,1,Urban,Y
LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349,360,1,Semiurban,N


In [0]:
data_loan.printSchema()
print((data_loan.count(), len(data_loan.columns)))

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)

(614, 13)


### groupby

In [0]:
data_loan.groupBy("Education").min("ApplicantIncome").show()
data_loan.groupBy("Married").avg("CoapplicantIncome").show()
data_loan.groupBy("Loan_Status").count().show()

+------------+--------------------+
|   Education|min(ApplicantIncome)|
+------------+--------------------+
|Not Graduate|                 210|
|    Graduate|                 150|
+------------+--------------------+

+-------+----------------------+
|Married|avg(CoapplicantIncome)|
+-------+----------------------+
|   null|    251.33333333333334|
|     No|    1316.5586854460093|
|    Yes|     1794.632964795578|
+-------+----------------------+

+-----------+-----+
|Loan_Status|count|
+-----------+-----+
|          Y|  422|
|          N|  192|
+-----------+-----+



In [0]:

data_loan.groupBy("Education","Loan_Status","Married").min("CoapplicantIncome").show()
data_loan.groupBy("Loan_Status","Education").mean("ApplicantIncome").show()
data_loan.groupBy("Married","Education").count().show()

+------------+-----------+-------+----------------------+
|   Education|Loan_Status|Married|min(CoapplicantIncome)|
+------------+-----------+-------+----------------------+
|Not Graduate|          Y|     No|                   0.0|
|    Graduate|          N|    Yes|                   0.0|
|Not Graduate|          N|    Yes|                   0.0|
|Not Graduate|          N|     No|                   0.0|
|    Graduate|          N|     No|                   0.0|
|    Graduate|          Y|   null|                   0.0|
|Not Graduate|          Y|    Yes|                   0.0|
|    Graduate|          Y|    Yes|                   0.0|
|    Graduate|          Y|     No|                   0.0|
+------------+-----------+-------+----------------------+

+-----------+------------+--------------------+
|Loan_Status|   Education|avg(ApplicantIncome)|
+-----------+------------+--------------------+
|          Y|    Graduate|   5751.576470588236|
|          N|Not Graduate|  3646.4423076923076|
|    

### drop null()

In [0]:
loan_copy = data_loan
loan_clean = loan_copy.na.drop()


### data after dropping null

In [0]:
print((data_loan.count(), len(data_loan.columns)))
print((loan_clean.count(), len(loan_clean.columns)))

(614, 13)
(480, 13)


### groupby after null drop

In [0]:
loan_clean.groupBy("Education").min("ApplicantIncome").show()
loan_clean.groupBy("Married").avg("CoapplicantIncome").show()
loan_clean.groupBy("Loan_Status").count().show()

loan_clean.groupBy("Education","Loan_Status","Married").min("CoapplicantIncome").show()
loan_clean.groupBy("Loan_Status","Education").mean("ApplicantIncome").show()
loan_clean.groupBy("Married","Education").count().show()

+------------+--------------------+
|   Education|min(ApplicantIncome)|
+------------+--------------------+
|Not Graduate|                1442|
|    Graduate|                 150|
+------------+--------------------+

+-------+----------------------+
|Married|avg(CoapplicantIncome)|
+-------+----------------------+
|     No|    1215.8934911242604|
|    Yes|    1779.5463665229581|
+-------+----------------------+

+-----------+-----+
|Loan_Status|count|
+-----------+-----+
|          Y|  332|
|          N|  148|
+-----------+-----+

+------------+-----------+-------+----------------------+
|   Education|Loan_Status|Married|min(CoapplicantIncome)|
+------------+-----------+-------+----------------------+
|Not Graduate|          Y|     No|                   0.0|
|    Graduate|          N|    Yes|                   0.0|
|Not Graduate|          N|    Yes|                   0.0|
|Not Graduate|          N|     No|                   0.0|
|    Graduate|          N|     No|                   0.0|

# Joins

### Data creation

In [0]:
from pyspark.sql import SparkSession


# Initialize SparkSession
spark = SparkSession.builder \
.appName("example") \
.getOrCreate()
# Data
emp = [(1,"Smith",-1,"2018","10","M",3000),(2, "Rose",1 , "2010", "20","M", 4000),(3,"Williams",1,"2010","10","M",1000),(4, "Jones",2 ,"2005","10","F",2000),(5,"Brown",2,"2010","40","",-1),(6, "Brown", 2, "2010","50","",-1)]
empColumns = ["emp_id","name","superior_emp_id","year_joined", "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show()

dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     5|   Brown|              2|       2010|         40|      |    -1|
|     6|   Brown|              2|       2010|         50|      |    -1|
+------+--------+---------------+-----------+-----------+------+-----

### Inner, Outer, Full

In [0]:
#Inner join
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "inner").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#outer join
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "outer").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  null|    null|           null|       null|       null|  null|  null|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     null|   null|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#full join
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "full").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  null|    null|           null|       null|       null|  null|  null|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     null|   null|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### left, left outer

In [0]:
#Left join
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "left").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     null|   null|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#Left join
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "leftouter").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     null|   null|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### Right, Right Outer

In [0]:
#right join
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "right").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  null|    null|           null|       null|       null|  null|  null|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#right outer join
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "rightouter").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  null|    null|           null|       null|       null|  null|  null|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### LeftSemi LeftAnti

In [0]:
#leftsemijoin
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "leftsemi").show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     5|   Brown|              2|       2010|         40|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



In [0]:
#leftanti
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, "leftanti").show()

+------+-----+---------------+-----------+-----------+------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|     6|Brown|              2|       2010|         50|      |    -1|
+------+-----+---------------+-----------+-----------+------+------+



# Spark SQL using Loan Data (Clean)

### Create View

In [0]:
loan_clean.createOrReplaceTempView("loan")
loan_clean.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



###  Select query

In [0]:
spark.sql("select * from loan").limit(10).display()
loan_clean.select("Loan_ID","ApplicantIncome","Loan_Status").show(5)

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95,360,1,Urban,Y
LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158,360,0,Semiurban,N
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168,360,1,Urban,Y
LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349,360,1,Semiurban,N
LP001024,Male,Yes,2,Graduate,No,3200,700.0,70,360,1,Urban,Y


+--------+---------------+-----------+
| Loan_ID|ApplicantIncome|Loan_Status|
+--------+---------------+-----------+
|LP001003|           4583|          N|
|LP001005|           3000|          Y|
|LP001006|           2583|          Y|
|LP001008|           6000|          Y|
|LP001011|           5417|          Y|
+--------+---------------+-----------+
only showing top 5 rows



### filter based on data

In [0]:
spark.sql("""SELECT * From loan WHERE ApplicantIncome<5000""").limit(10).display()

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95,360,1,Urban,Y
LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158,360,0,Semiurban,N
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168,360,1,Urban,Y
LP001024,Male,Yes,2,Graduate,No,3200,700.0,70,360,1,Urban,Y
LP001028,Male,Yes,2,Graduate,No,3073,8106.0,200,360,1,Urban,Y
LP001029,Male,No,0,Graduate,No,1853,2840.0,114,360,1,Rural,N
LP001030,Male,Yes,2,Graduate,No,1299,1086.0,17,120,1,Urban,Y


### order by

In [0]:
spark.sql("""select * FROM loan WHERE Dependents in ('3+','2')order by Dependents """).limit(10).display()

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001199,Male,Yes,2,Not Graduate,No,3357,2859.0,144,360,1,Urban,Y
LP001491,Male,Yes,2,Graduate,Yes,3316,3500.0,88,360,1,Urban,Y
LP001245,Male,Yes,2,Not Graduate,Yes,1875,1875.0,97,360,1,Semiurban,Y
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168,360,1,Urban,Y
LP001267,Female,Yes,2,Graduate,No,1378,1881.0,167,360,1,Urban,N
LP001024,Male,Yes,2,Graduate,No,3200,700.0,70,360,1,Urban,Y
LP001318,Male,Yes,2,Graduate,No,6250,5654.0,188,180,1,Semiurban,Y
LP001030,Male,Yes,2,Graduate,No,1299,1086.0,17,120,1,Urban,Y
LP001319,Male,Yes,2,Not Graduate,No,3273,1820.0,81,360,1,Urban,Y
LP001179,Male,Yes,2,Graduate,No,4616,0.0,134,360,1,Urban,N


### group by

In [0]:
spark.sql("""SELECT Loan_Amount_Term,count(*) as count FROM loan GROUP BY Loan_Amount_Term""").show()

+----------------+-----+
|Loan_Amount_Term|count|
+----------------+-----+
|             300|    9|
|             360|  411|
|             120|    3|
|              84|    3|
|             240|    2|
|             480|   12|
|              60|    2|
|             180|   36|
|              36|    2|
+----------------+-----+

